# Converting event data to event-centric triples and adding them to the Knowledge Graph

We load the event data created by ChatGPT and convert these to event-centric triples. We push the triples to the Knowledge Graph.

In [8]:
import json
import pickle
import requests
import random
from datetime import date, datetime
from cltl.brain.long_term_memory import LongTermMemory
from cltl.brain.utils.helper_functions import brain_response_to_json
from cltl.commons.discrete import UtteranceType
from tqdm import tqdm
#from datetime import date, datetime
from random import getrandbits
from pathlib import Path

## Loading the ChatGPT event and semantic role annotations 

In [50]:
from llm_event_triples_openai import LLM_EventExtraction
# import pickle
# path = "./diabetes/event_data.pkl"
# dbfile = open(path, 'rb')    
# all_annotations = pickle.load(dbfile)

f = open("./diabetes/event_srl.json", "r")
f = open("/Users/piek/Downloads/test.json", "r")
f = open("./diabetes/event_srl_rest.json", "r")
annotated_conversations = json.load(f)

In [51]:
print(len(annotated_conversations))
print(annotated_conversations[0])

22
[{'chat': 234, 'date': '2010,Jun,09', 'human': 'Ali', 'Input': {'turn': 1, 'speaker': 'Ali', 'utterance': "Lately, I've noticed I'm losing weight even though I haven't changed my diet or exercise routine. Could this be related to my diabetes?"}, 'Output': {'activity': 'Unexplained weight loss', 'agent': ['Ali'], 'patient': ["Ali's body"], 'instrument': [], 'manner': [], 'location': [], 'time': 'Lately'}}, {'chat': 234, 'date': '2010,Jun,09', 'human': 'Ali', 'Input': {'turn': 3, 'speaker': 'Ali', 'utterance': 'I see. Should I be worried and inform my doctor immediately?'}, 'Output': {'activity': 'Inform healthcare provider', 'agent': ['Ali'], 'patient': ['healthcare provider'], 'instrument': [], 'manner': [], 'location': [], 'time': 'immediately'}}, {'chat': 234, 'date': '2010,Jun,09', 'human': 'Ali', 'Input': {'turn': 5, 'speaker': 'Ali', 'utterance': 'In the meantime, is there anything I can do to manage this?'}, 'Output': {'activity': 'monitor blood sugar levels', 'agent': ['Ali']

In [52]:
# for conversation in annotated_conversations:
#     if len(conversation)>0:
#         for turn in conversation:
#             if 'Output' in turn and not turn['Output']==None:
#                 if 'activity' in turn['Output']:
#                     print(turn['Output']['activity'])
#                 else:
#                     print("No output")

## Converting event data to capsules with event-centric triples

In [53]:
import events_to_capsules
from dateutil import parser

# Define contextual features
context_id = getrandbits(8)
place_id = getrandbits(8)
location = requests.get("https://ipinfo.io").json()

In [54]:
scenarios = []
for conversation in annotated_conversations:
    if len(conversation)>0:
        scenario_context = {"context_id": conversation[0]['chat'],
             "date": parser.parse(conversation[0]['date']),
             "place": "Piek's office",
             "place_id": place_id,
             "country": location['country'],
             "region": location['region'],
             "city": location['city']}
        capsules = []
        for turn in conversation:
            turn_capsules = events_to_capsules.get_capsules_from_turn(turn)
            capsules.extend(turn_capsules)
        if capsules:
            scenario = (scenario_context, capsules)
            scenarios.append(scenario)

In [55]:
for (context_capsule, content_capsules) in tqdm(scenarios):
    print(context_capsule['context_id'], len(content_capsules))

100%|██████████████████████████████████████| 22/22 [00:00<00:00, 40667.56it/s]

234 16
235 20
236 21
237 17
238 17
239 20
240 32
241 17
242 12
243 16
244 24
245 19
246 13
247 24
248 21
249 20
250 25
251 17
252 11
253 17
254 20
255 18


## Adding triples to the episodic Knowledge Graph

In [56]:
def main(scenarios):
    # Create folders
    scenario_filepath = Path('./diabetes/')
    graph_filepath = scenario_filepath / Path('graph/')
    graph_filepath.mkdir(parents=True, exist_ok=True)

    # Create brain connection
    brain = LongTermMemory(address="http://localhost:7200/repositories/diabetes",  # Location to save accumulated graph
                           log_dir=graph_filepath,  # Location to save step-wise graphs
                           clear_all=False)  # To start from an empty brain

    # Loop through capsules
    all_capsules = []
    all_responses = []
    for (context_capsule, content_capsules) in tqdm(scenarios):
        #print(f"\n\n---------------------------------------------------------------\n")

        print(context_capsule['context_id'], len(content_capsules))        # Create context
        response = brain.capsule_context(context_capsule)

        # Add information to the brain
        for capsule in content_capsules:
            print('content_capsule', capsule["author"])
            if capsule['utterance_type'] == UtteranceType.STATEMENT:
              #  print(capsule)
                response = brain.capsule_statement(capsule, reason_types=True, return_thoughts=False, create_label=True)
              #  print(f"\nTriple: {capsule['triple']}\n")
            if capsule['utterance_type'] == UtteranceType.EXPERIENCE:
                response = brain.capsule_experience(capsule, create_label=True)
                print(f"\nEntity: {capsule['entity']}\n")

            # capsule_json = brain_response_to_json(capsule)
            # all_capsules.append(capsule_json)
            # response_json = brain_response_to_json(response)
            # all_responses.append(response_json)

    # Save responses 
    f = open(scenario_filepath / "capsules.json", "w")
    json.dump(all_capsules, f)
    f = open(scenario_filepath / "responses.json", "w")
    json.dump(all_responses, f)

In [57]:
main(scenarios)

2025-08-28 15:48:47 -     INFO -                                    cltl.brain.LongTermMemory - Booted
2025-08-28 15:48:47 -     INFO -                                  cltl.brain.ThoughtGenerator - Booted
2025-08-28 15:48:47 -     INFO -                                  cltl.brain.LocationReasoner - Booted
2025-08-28 15:48:47 -     INFO -                                      cltl.brain.TypeReasoner - Booted
2025-08-28 15:48:47 -     INFO -                                   cltl.brain.TrustCalculator - Booted
  0%|                                                  | 0/22 [00:00<?, ?it/s]2025-08-28 15:48:47 -     INFO -                                    cltl.brain.LongTermMemory - Context: context234


234 16


2025-08-28 15:48:48 -     INFO -                                    cltl.brain.LongTermMemory - Triple in statement: unexplained-weight-loss_agent_ali [activity_->_agent])


content_capsule {'label': 'Ali', 'type': ['agent'], 'uri': 'http://cltl.nl/leolani/friends/Ali'}


2025-08-28 15:48:52 -     INFO -                                    cltl.brain.LongTermMemory - Triple in statement: unexplained-weight-loss_patient_ali-s-body [activity_->_agent or object])


...

2025-08-28 15:54:47 -     INFO -                                    cltl.brain.LongTermMemory - Triple in statement: provides-suggestions-on-managing-blood-sugar-portion-control-and-importance-of-mediterranean-diet_instrument_dietary-suggestions [activity_->_instrument])


content_capsule {'label': 'Mehmet', 'type': ['agent'], 'uri': 'http://cltl.nl/leolani/friends/Mehmet'}


2025-08-28 15:54:48 -     INFO -                                    cltl.brain.LongTermMemory - Triple in statement: provides-suggestions-on-managing-blood-sugar-portion-control-and-importance-of-mediterranean-diet_time_during-a-medical-consultation [activity_->_time])


content_capsule {'label': 'Mehmet', 'type': ['agent'], 'uri': 'http://cltl.nl/leolani/friends/Mehmet'}


100%|█████████████████████████████████████████| 22/22 [06:02<00:00, 16.47s/it]


# End of notebook